In [1]:
import pandas as pd
import numpy as np
import urllib, json, os, datetime, time
from unidecode import unidecode
import unicodecsv as csv

In [2]:
ROOT_DIR = "../../"
with open(ROOT_DIR + "config/config-scraping.json") as f:
    cfg = json.load(f)

In [4]:
import requests

In [5]:
#CHANGE THESE SETTINGS!
save_dir = ROOT_DIR + cfg["tmp_dir"]
board = "wg"

print ("The current working directory is " + save_dir)  

#First we'll try to create a folder for the board.
path = save_dir + board
if not os.path.exists(path):
    os.mkdir(path)

#Get the 4chan board catalog JSON file and open it
url = "https://a.4cdn.org/" + board + "/catalog.json"
response = requests.get(url)
data = eval(response.content)

The current working directory is ../../data/tmp/


In [6]:
def get_threads(data):
    threads = []
    for page in data:
        for thread in page["threads"]:
            threads.append(thread)
            
    return threads

In [7]:
threads_df = pd.DataFrame(get_threads(data))

In [8]:
def get_thread_posts(thread_no, board):
    time.sleep(1)
    response = requests.get("https://a.4cdn.org/" + board + "/thread/" + str(thread_no) + ".json")
    if response.ok:
        return eval(response.content)["posts"]
    else:
        return None

In [9]:
threads = []
for thread_no in threads_df["no"]:
    threads.append(get_thread_posts(thread_no, board))

In [10]:
def get_thread_images(posts):
    images = [".jpg", ".png"]
    media = [str(post["tim"])+post["ext"] for post in posts if "tim" in post]
    return list(filter(lambda im:im[-4:] in images, media))

In [11]:
images = []
for thread in threads:
    images += get_thread_images(thread)

In [12]:
def download_images(images, board, save_dir):
    if len(images) > 60:
        if len(images) > 3600:
            print("Minimum time to completion: {} hours".format(len(images) / 3600))
        else:
            print("Minimum time to completion: {} minutes".format(len(images) / 60))
            
    im_count = 0
    for image in images:
        # API rules ask for a 1 second delay between requests
        time.sleep(1)
        try:
            img = requests.get("https://i.4cdn.org/{}/{}".format(board, image)).content
            with open(save_dir + board + '/' + image, "wb") as f:
                f.write(img)
        except:
            print("Error with image: {}".format(image))
            continue
            
        im_count += 1
        if im_count == 1:
            print("Saved {} image".format(im_count), end='\r')
        else:
            print("Saved {} images".format(im_count), end='\r')

- Download thumbnails for image analysis
- Maintain original image resolution for resolution visualization
- Should probably hold off on downloading any images until after metadata filtering is done
- Add nsfw tag
    - https://github.com/yahoo/open_nsfw

In [53]:
def get_complete_df(threads):
    posts = []
    thread_num = 0
    for thread in threads:
        for post in thread:
            post["thread_id"] = thread_num
        posts += thread
        thread_num += 1
        
    return posts

In [54]:
df = pd.DataFrame(get_complete_df(threads))

In [69]:
df.keys()

Index(['no', 'sticky', 'closed', 'now', 'name', 'sub', 'com', 'filename',
       'ext', 'w', 'h', 'tn_w', 'tn_h', 'tim', 'time', 'md5', 'fsize', 'resto',
       'semantic_url', 'replies', 'images', 'unique_ips', 'thread_id',
       'bumplimit', 'imagelimit', 'trip', 'filedeleted'],
      dtype='object')

In [77]:
# Storage usage
df.fsize.sum()/1000000000

12.534209289

In [74]:
df[(df.w * df.h) >= (2560 * 1080)]

,no,sticky,closed,now,name,sub,com,filename,ext,w,...,resto,semantic_url,replies,images,unique_ips,thread_id,bumplimit,imagelimit,trip,filedeleted
1,6872255,NaN,NaN,03\/11\/17(Sat)16:11:19,The \/wg\/ Bros,NaN,Desktop \/ Theme Hax Guide<br><br>ALL THE LINK...,deskhax,.jpg,3600.0,...,6872254,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
2,6872256,NaN,NaN,03\/11\/17(Sat)16:12:28,The \/wg\/ Bros,NaN,Mac Theme Hax<br><br>Here&#039;s to OS X<br>~ ...,machax,.jpg,3200.0,...,6872254,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
4,6872260,NaN,NaN,03\/11\/17(Sat)16:14:36,The \/wg\/ Bros,NaN,~New Visitors Please Read~<br><br>This guide w...,newcomer guide,.png,3000.0,...,6872254,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
5,7565401,NaN,NaN,03\/20\/20(Fri)12:42:56,Anonymous,NaN,No Non-Fiction Space Thread?,5b9j0qcz8mwx,.png,3533.0,...,0,no-nonfiction-space-thread,130.0,94.0,47.0,1,0.0,0.0,NaN,NaN
6,7565403,NaN,NaN,03\/20\/20(Fri)12:45:29,Anonymous,NaN,NaN,1583857960812,.jpg,5120.0,...,7565401,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12589,7554805,NaN,NaN,02\/24\/20(Mon)15:56:02,Anonymous,NaN,NaN,TVRC%2002491,.jpg,3072.0,...,7552194,NaN,NaN,NaN,NaN,149,NaN,NaN,NaN,NaN
12594,7560104,NaN,NaN,03\/08\/20(Sun)00:43:31,Anonymous,NaN,BRAND NEW,Ground 0,.png,2560.0,...,7552194,NaN,NaN,NaN,NaN,149,NaN,NaN,NaN,NaN
12607,7569872,NaN,NaN,03\/29\/20(Sun)11:22:10,Anonymous,NaN,NaN,IndianHeadTestPattern16x9,.png,3456.0,...,7552194,NaN,NaN,NaN,NaN,149,NaN,NaN,NaN,NaN
12613,7576424,NaN,NaN,04\/10\/20(Fri)14:42:31,Anonymous,NaN,"<a href=""#p7576315"" class=""quotelink"">&gt;&gt;...",Fantasy Test Card,.png,2560.0,...,7552194,NaN,NaN,NaN,NaN,149,NaN,NaN,NaN,NaN
